Reference:
- https://spacy.io/docs/usage/

In [1]:
import spacy

# Load Language engine

## First, you need to select Language and load engine of the language. This might take time (~15s).
- If you are using spacy for the first time, you have to download via `python -m spacy download en`.
- You can customize your pipeline via giving custom modules via keyword arguments

In [2]:
nlp = spacy.load('en')

In [1]:
sentence = 'New York city is located east side of America.'

## That's it! 'nlp' engine already processed all the nlp stuffs

In [4]:
nlp(sentence)

New York city is located east side of America.

# Brief overview of Tagging schema

In [5]:
for token in nlp(sentence):
    print(token)

New
York
city
is
located
east
side
of
America
.


In [7]:
token = nlp(sentence)[0]
token

New

In [8]:
type(token)

spacy.tokens.token.Token

In [9]:
token.lemma_

'new'

In [10]:
token.is_digit

False

In [11]:
token.is_alpha

True

In [12]:
token.is_oov

False

In [13]:
token.is_title

True

In [14]:
token.is_stop

False

In [15]:
for token in nlp(sentence):
    print(token, token.text)

New New
York York
city city
is is
located located
east east
side side
of of
America America
. .


## Tag

In [17]:
for token in nlp(sentence):
    print(token, token.tag_)

New NNP
York NNP
city NN
is VBZ
located VBN
east JJ
side NN
of IN
America NNP
. .


## Part-of-speech

In [18]:
for word in nlp(sentence):
    print(word.text, word.pos_)

New PROPN
York PROPN
city NOUN
is VERB
located VERB
east ADJ
side NOUN
of ADP
America PROPN
. PUNCT


## All at once!

In [19]:
for word in nlp(sentence):
    print(word.text, word.lemma, word.lemma_, word.tag, word.tag_, word.pos, word.pos_)

New 652 new 441 NNP 93 PROPN
York 19784 york 441 NNP 93 PROPN
city 1210 city 440 NN 89 NOUN
is 488 be 459 VBZ 97 VERB
located 14550 locate 457 VBN 97 VERB
east 3462 east 433 JJ 82 ADJ
side 850 side 440 NN 89 NOUN
of 471 of 432 IN 83 ADP
America 6413 america 441 NNP 93 PROPN
. 419 . 419 . 94 PUNCT


# Vocabulary

In [20]:
nlp.vocab

In [21]:
nlp.vocab.lang

'en'

In [22]:
len(nlp.vocab) # vocab size

1297536

In [21]:
nlp.vocab.length # vocab size

1297536

In [23]:
banana = nlp.vocab['banana']

In [24]:
banana.lower_

'banana'

# Word Vectors

- Default: GloVe
- If you need to train a word2vec model, I recommend Gensim.

In [25]:
sentence

'New York city is located east side of America.'

In [27]:
for token in nlp(sentence):
    print(token, len(token.vector))

New 300
York 300
city 300
is 300
located 300
east 300
side 300
of 300
America 300
. 300


In [25]:
for token in nlp('apple, peach, banana and orange'):
    print(token)

apple
,
peach
,
banana
and
orange


In [28]:
apple, _, peach, _, banana, and_, orange = nlp('apple, peach, banana and orange')

In [29]:
apple.vector.shape

(300,)

In [30]:
apple.similarity(peach)

0.27504403340499511

In [31]:
peach.similarity(apple)

0.27504403340499511

In [32]:
apple.similarity(orange)

0.22885475784967607

In [33]:
peach.similarity(banana)

0.3370778377992128

In [34]:
apple.similarity(and_)

0.088770645850664423

## The most similar words of given word!
- find words whose vectors are similar to given word's

In [35]:
def most_similar(word):
    queries = [w for w in word.vocab if w.is_lower == word.is_lower and w.prob >= -15]
    by_similarity = sorted(queries, key=lambda w: word.similarity(w), reverse=True)
    top_ten = by_similarity[:10]
    return [lexeme.lower_ for lexeme in top_ten]

In [36]:
most_similar(nlp.vocab['apple'])

['apple',
 'blackberry',
 'doritos',
 'asus',
 'onion',
 'apples',
 'oreo',
 'melon',
 'mango',
 'dell']

In [37]:
most_similar(nlp.vocab['ipad'])

['ipad',
 'iphone',
 'ipod',
 'ios',
 'ps3',
 'smartphone',
 'dreamcast',
 'netbook',
 'smartphones',
 'xbox']

In [36]:
most_similar(nlp.vocab['reddit'])

['reddit',
 'digg',
 'tumblr',
 'vimeo',
 'deviantart',
 'github',
 'craigslist',
 'instagram',
 'pinterest',
 'wordpress']

In [37]:
most_similar(nlp.vocab['fuck'])

['fuck',
 'bugger',
 'shit',
 'nigga',
 'fucking',
 'goddamn',
 'piss',
 'whoop',
 "c'mon",
 'whoa']

# Spacy processes multiple sentences at once!

In [38]:
sentences = "I'd like to drink some beer. Why do you learn deep learning? IPad is awesome."

In [39]:
doc = nlp(sentences)

In [40]:
list(doc.sents)

[I'd like to drink some beer.,
 Why do you learn deep learning?,
 IPad is awesome.]

# Parsing
- visualizer:
    - https://demos.explosion.ai/displacy/
    - https://github.com/explosion/displacy

In [41]:
doc = nlp('London is a big city in the United Kingdom.')
doc

London is a big city in the United Kingdom.

In [45]:
parsed_tokens = []
for token in doc:
    parsed_tokens.append([token, token.pos_, token.dep_, token.head, token.head.pos_])

In [46]:
import pandas as pd

In [47]:
columns = ['Word', 'POS', 'Dependency', 'head', 'POS of head']

In [48]:
pd.DataFrame(parsed_tokens, columns=columns)

,Word,POS,Dependency,head,POS of head
0,London,PROPN,nsubj,is,VERB
1,is,VERB,ROOT,is,VERB
2,a,DET,det,city,NOUN
3,big,ADJ,amod,city,NOUN
4,city,NOUN,attr,is,VERB
5,in,ADP,prep,city,NOUN
6,the,DET,det,Kingdom,PROPN
7,United,PROPN,compound,Kingdom,PROPN
8,Kingdom,PROPN,pobj,in,ADP
9,.,PUNCT,punct,is,VERB


### Visualization
- [word-level](https://demos.explosion.ai/displacy/?text=London%20is%20a%20big%20city%20in%20the%20United%20Kingdom.&model=en&cpu=1&cph=1)
- [phrase-level](https://demos.explosion.ai/displacy/?text=London%20is%20a%20big%20city%20in%20the%20United%20Kingdom.&model=en&cpu=1&cph=0)

### Traversing tree

In [46]:
from spacy.symbols import nsubj, VERB

In [47]:
verbs = set()

In [48]:
for token in doc:
    if token.dep == nsubj and token.head.pos == VERB:
        verbs.add(token.head)

In [49]:
verbs

{is}

# Entity Recognition

IOB formatting (In, Begin, Out).

CONLL 2002
- geo = Geographical Entity
- org = Organization
- per = Person
- gpe = Geopolitical Entity
- tim = Time indicator
- art = Artifact
- eve = Event
- nat = Natural Phenomenon

In [50]:
for token in nlp('London is a big city in the United Kingdom.'):
    print(token.text, token.ent_iob, token.ent_type_) # GPE = Geopolitical Entity

London 3 GPE
is 2 
a 2 
big 2 
city 2 
in 2 
the 3 GPE
United 1 GPE
Kingdom 1 GPE
. 2 
